In [1]:
import scanpy as sc
import anndata as ad
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
with open("/Users/alicedriessen/Library/CloudStorage/Box-Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/functional_and_score_genes.txt") as f:
    genes = f.readlines()
genes = [g.strip() for g in genes]
len(genes)

82

In [3]:
with open("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CAR_variants.txt") as f:
    cars = f.readlines()
cars = [c.strip() for c in cars]

# chemCPA

In [4]:
cd4 = sc.read_h5ad("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD4.h5ad")
cd8 = sc.read_h5ad("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD8.h5ad")

In [5]:
cd4 = cd4[:, genes]
cd8 = cd8[:, genes]

cd4.uns["all_genes"] = {f"{c}_{d}": genes for c in cars for d in ['D09', 'D17']}
cd8.uns["all_genes"] = {f"{c}_{d}": genes for c in cars for d in ['D09', 'D17']}

cd4.obs["dose"] = 1
cd8.obs["dose"] = 1

cd4.obs["car_donor"] = cd4.obs["CAR_Variant"].astype(str) + "_" + cd4.obs["Donor"].astype(str)
cd8.obs["car_donor"] = cd8.obs["CAR_Variant"].astype(str) + "_" + cd8.obs["Donor"].astype(str)

cd4.obs["dummy_donor"] = "D09"
cd8.obs["dummy_donor"] = "D09"

cd4.obs["car_dummydonor"] = cd4.obs["CAR_Variant"].astype(str) + "_" + cd4.obs["dummy_donor"].astype(str)
cd8.obs["car_dummydonor"] = cd8.obs["CAR_Variant"].astype(str) + "_" + cd8.obs["dummy_donor"].astype(str)

cd4

/var/folders/02/gztwz1tj1051mtt_ykv6v7940000gn/T/ipykernel_59513/1829474723.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  cd4.uns["all_genes"] = {f"{c}_{d}": genes for c in cars for d in ['D09', 'D17']}
/var/folders/02/gztwz1tj1051mtt_ykv6v7940000gn/T/ipykernel_59513/1829474723.py:5: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  cd8.uns["all_genes"] = {f"{c}_{d}": genes for c in cars for d in ['D09', 'D17']}


AnnData object with n_obs × n_vars = 26077 × 82
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'CAR_Variant', 'Donor', 'Time', 'Lane', 'Sample', 'percent.mt', 'A', 'B', 'C', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'RNA_snn_res.0.75', 'seurat_clusters', 'subset_RNA', 'subset_ADT', 'subset', 'Cytotoxicity_1', 'Proinflamatory_2', 'Memory_3', 'CD4_Th1_4', 'CD4_Th2_5', 'ident', 'time_subset', 'dose', 'car_donor', 'dummy_donor', 'car_dummydonor'
    uns: 'all_genes'
    obsm: 'HARMONY', 'X_pca', 'X_umap'
    layers: 'logcounts'

## Monge setting - ID 0.8/0.2 train/test split
### One monge model per CAR variant

In [6]:
for car in cars:
    sel_cars = [car] + ["NA-NA-NA"]
    train_idx, test_idx = train_test_split(cd4[cd4.obs.CAR_Variant.isin(sel_cars)].obs_names, train_size=0.8, random_state=1234,
                                          stratify=cd4[cd4.obs.CAR_Variant.isin(sel_cars)].obs.CAR_Variant)
    cd4.obs.loc[train_idx, f"{car}_ID"] = "train"
    cd4.obs.loc[test_idx, f"{car}_ID"] = "test"

    train_idx, test_idx = train_test_split(cd8[cd8.obs.CAR_Variant.isin(sel_cars)].obs_names, train_size=0.8, random_state=1234,
                                          stratify=cd8[cd8.obs.CAR_Variant.isin(sel_cars)].obs.CAR_Variant)
    cd8.obs.loc[train_idx, f"{car}_ID"] = "train"
    cd8.obs.loc[test_idx, f"{car}_ID"] = "test"

In [7]:
cd4.obs.groupby(["CAR_Variant", "CTLA4-CD40-z_ID"], observed=True).size()

CAR_Variant   CTLA4-CD40-z_ID
CTLA4-CD40-z  test                41
              train              162
NA-NA-NA      test               152
              train              607
dtype: int64

In [8]:
cd8.obs.groupby(["CAR_Variant", "Donor", "CTLA4-CTLA4-z_ID"], observed=True).size()

CAR_Variant    Donor  CTLA4-CTLA4-z_ID
CTLA4-CTLA4-z  D09    test                 19
                      train                66
               D17    test                  5
                      train                32
NA-NA-NA       D09    test                152
                      train               545
               D17    test                133
                      train               593
dtype: int64

In [9]:
cd8.obs.groupby(["CAR_Variant", "CD40-CTLA4-z_ID"], observed=True).size()

CAR_Variant   CD40-CTLA4-z_ID
CD40-CTLA4-z  test                140
              train               559
NA-NA-NA      test                285
              train              1138
dtype: int64

In [10]:
cd8.obs.groupby(["CAR_Variant", "CD40-IL15RA-z_ID"], observed=True).size()

CAR_Variant    CD40-IL15RA-z_ID
CD40-IL15RA-z  test                 138
               train                553
NA-NA-NA       test                 285
               train               1138
dtype: int64

In [11]:
559/32

17.46875

## CMonge setting - ID 0.8/0.2 train/test split
### One CMonge model for all CARs with enough cells

In [12]:
with open("/Users/alicedriessen/Box/CAR_Tcells/Model/OT/CD4_sel_cars.txt") as f:
    cars_cd4 = f.readlines()
cars_cd4 = [c.strip() for c in cars_cd4]+ ["NA-NA-NA"]
cars_cd4

with open("/Users/alicedriessen/Box/CAR_Tcells/Model/OT/CD8_sel_cars.txt") as f:
    cars_cd8 = f.readlines()
cars_cd8 = [c.strip() for c in cars_cd8] + ["NA-NA-NA"]
cars_cd8

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alicedriessen/Box/CAR_Tcells/Model/OT/CD4_sel_cars.txt'

In [ ]:
train_idx, test_idx = train_test_split(cd4[cd4.obs.CAR_Variant.isin(cars_cd4)].obs_names, train_size=0.8, random_state=1234, 
                                       stratify=cd4[cd4.obs.CAR_Variant.isin(cars_cd4)].obs.CAR_Variant)
cd4.obs.loc[train_idx, "sel_CARs_ID"] = "train"
cd4.obs.loc[test_idx, "sel_CARs_ID"] = "test"

train_idx, test_idx = train_test_split(cd8[cd8.obs.CAR_Variant.isin(cars_cd8)].obs_names, train_size=0.8, random_state=1234, 
                                       stratify=cd8[cd8.obs.CAR_Variant.isin(cars_cd8)].obs.CAR_Variant)
cd8.obs.loc[train_idx, "sel_CARs_ID"] = "train"
cd8.obs.loc[test_idx, "sel_CARs_ID"] = "test"

In [ ]:
cd4.obs.groupby(["CAR_Variant", "sel_CARs_ID"], observed=True).size()

In [ ]:
cd8.obs.groupby(["CAR_Variant", "sel_CARs_ID"], observed=True).size()

## CMonge OOD setting - OOD - Can use pretrained model from CMonge setting
### Otherwise include low cell number CARs as OOD above

In [ ]:
train_idx, test_idx = train_test_split(cd4[cd4.obs.CAR_Variant.isin(cars_cd4)].obs_names, train_size=0.8, random_state=1234, 
                                       stratify=cd4[cd4.obs.CAR_Variant.isin(cars_cd4)].obs.CAR_Variant)
cd4.obs.loc[train_idx, "sel_CARs_OOD"] = "train"
cd4.obs.loc[test_idx, "sel_CARs_OOD"] = "test"
cd4.obs.loc[~cd4.obs.CAR_Variant.isin(cars_cd4), "sel_CARs_OOD"] = "ood"

train_idx, test_idx = train_test_split(cd8[cd8.obs.CAR_Variant.isin(cars_cd8)].obs_names, train_size=0.8, random_state=1234, 
                                       stratify=cd8[cd8.obs.CAR_Variant.isin(cars_cd8)].obs.CAR_Variant)
cd8.obs.loc[train_idx, "sel_CARs_OOD"] = "train"
cd8.obs.loc[test_idx, "sel_CARs_OOD"] = "test"
cd8.obs.loc[~cd8.obs.CAR_Variant.isin(cars_cd8), "sel_CARs_OOD"] = "ood"



In [ ]:
cd4.obs.groupby(["CAR_Variant", "sel_CARs_OOD"], observed=True).size()

## 41BB OOD

In [ ]:
ood_cars = ["41BB-41BB-z", "41BB-CD28-z", "41BB-CD40-z", "41BB-CTLA4-z", "41BB-IL15RA-z", 
            "41BB-NA-z", "CD28-41BB-z", "CD40-41BB-z", "CTLA4-41BB-z", "IL15RA-41BB-z"]
id_cars = ["CD28-CD28-z", "CD28-CD40-z", "CD28-CTLA4-z", "CD28-IL15RA-z ", "CD28-NA-z", 
           "CD40-CD28-z", "CD40-CD40-z", "CD40-CTLA4-z", "CD40-IL15RA-z", "CD40-NA-z", 
           "CTLA4-CD28-z", "CTLA4-CD40-z", "CTLA4-CTLA4-z", "CTLA4-IL15RA-z", "CTLA4-NA-z", 
           "IL15RA-CD28-z", "IL15RA-CD40-z", "IL15RA-CTLA4-z", "IL15RA-IL15RA-z", "IL15RA-NA-z",
          "NA-NA-NA"]

In [ ]:
train_idx, test_idx = train_test_split(cd4[cd4.obs.CAR_Variant.isin(id_cars)].obs_names, train_size=0.8, random_state=1234, 
                                       stratify=cd4[cd4.obs.CAR_Variant.isin(id_cars)].obs.CAR_Variant)
cd4.obs.loc[train_idx, "41BB_OOD"] = "train"
cd4.obs.loc[test_idx, "41BB_OOD"] = "test"
cd4.obs.loc[cd4.obs.CAR_Variant.isin(ood_cars), "41BB_OOD"] = "ood"

train_idx, test_idx = train_test_split(cd8[cd8.obs.CAR_Variant.isin(id_cars)].obs_names, train_size=0.8, random_state=1234, 
                                       stratify=cd8[cd8.obs.CAR_Variant.isin(id_cars)].obs.CAR_Variant)
cd8.obs.loc[train_idx, "41BB_OOD"] = "train"
cd8.obs.loc[test_idx, "41BB_OOD"] = "test"
cd8.obs.loc[cd8.obs.CAR_Variant.isin(ood_cars), "41BB_OOD"] = "ood"


In [ ]:
cd8.obs.groupby(["CAR_Variant", "41BB_OOD"], observed=True).size()

## Indicate control

In [ ]:
# cd4.obs["chemcpa_car"] = ["control" if c=="NA-NA-NA" else c for c in cd4.obs.CAR_Variant]
# cd8.obs["chemcpa_car"] = ["control" if c=="NA-NA-NA" else c for c in cd8.obs.CAR_Variant]

cd4.obs["control"] = [1 if c=="NA-NA-NA" else 0 for c in cd4.obs.CAR_Variant]
cd8.obs["control"] = [1 if c=="NA-NA-NA" else 0 for c in cd8.obs.CAR_Variant]


## Make normalised data the default

In [ ]:
cd4.X = cd4.layers["logcounts"]
cd8.X = cd8.layers["logcounts"]

## Save data

In [ ]:
cd4.write_h5ad("/Users/alicedriessen/Box/CAR_Tcells/Model/OT/CD4_chemCPA_anno.h5ad")
cd8.write_h5ad("/Users/alicedriessen/Box/CAR_Tcells/Model/OT/CD8_chemCPA_anno.h5ad")

## Embedding

In [ ]:
esm = pd.read_csv("/Users/alicedriessen/Library/CloudStorage/Box-Box/CAR_Tcells/Model/OT/embedding/esm2_t48_15B_UR50D_tail_dim", index_col=0)

In [ ]:
esm.T.to_csv("/Users/alicedriessen/Library/CloudStorage/Box-Box/CAR_Tcells/Model/OT/embedding/esm2_t48_15B_UR50D_tail_dim_transposed")

# Average CAR comparison

In [5]:
cd4 = sc.read_h5ad("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD4.h5ad")
cd8 = sc.read_h5ad("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD8.h5ad")

In [6]:
with open("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD4_sel_cars.txt") as f:
    cd4_cars = f.readlines()
cd4_cars = [c.strip() for c in cd4_cars]
with open("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD8_sel_cars.txt") as f:
    cd8_cars = f.readlines()
cd8_cars = [c.strip() for c in cd8_cars]

In [7]:
cars = [c for c in cars if not c in ["NA-NA-NA", "NA-NA-z"]]

### CD4

In [8]:
cd4

AnnData object with n_obs × n_vars = 26077 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'CAR_Variant', 'Donor', 'Time', 'Lane', 'Sample', 'percent.mt', 'A', 'B', 'C', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'RNA_snn_res.0.75', 'seurat_clusters', 'subset_RNA', 'subset_ADT', 'subset', 'Cytotoxicity_1', 'Proinflamatory_2', 'Memory_3', 'CD4_Th1_4', 'CD4_Th2_5', 'ident', 'time_subset'
    obsm: 'HARMONY', 'X_pca', 'X_umap'
    layers: 'logcounts'

In [9]:
min_cells = cd4.obs.groupby("CAR_Variant").size().min()
sampled_adata = {}
for car in cars:
    sample = sc.pp.sample(cd4, n=min_cells, replace=False, copy=True, rng=1234)
    if car in cd4_cars:
        sample.obs["CAR_Variant"] = "sel_cars_mixed"
        cd4 = ad.concat([cd4, sample])
    if "41BB" in car:
        sample.obs["CAR_Variant"] = "41BB_OOD_cars_mixed"
        cd4 = ad.concat([cd4, sample])
    sample.obs["CAR_Variant"] = "all_cars_mixed"
    cd4 = ad.concat([cd4, sample])
cd4.obs_names_make_unique()

/var/folders/02/gztwz1tj1051mtt_ykv6v7940000gn/T/ipykernel_6316/1740129232.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  min_cells = cd4.obs.groupby("CAR_Variant").size().min()
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To mak

In [10]:
min_cells

190

In [11]:
cd4.obs["CAR_Variant"].unique()

array(['41BB-41BB-z', '41BB-CD28-z', '41BB-CD40-z', '41BB-CTLA4-z',
       '41BB-IL15RA-z', '41BB-NA-z', 'CD28-41BB-z', 'CD28-CD28-z',
       'CD28-CD40-z', 'CD28-CTLA4-z', 'CD28-IL15RA-z', 'CD28-NA-z',
       'CD40-41BB-z', 'CD40-CD28-z', 'CD40-CD40-z', 'CD40-CTLA4-z',
       'CD40-IL15RA-z', 'CD40-NA-z', 'CTLA4-41BB-z', 'CTLA4-CD28-z',
       'CTLA4-CD40-z', 'CTLA4-CTLA4-z', 'CTLA4-IL15RA-z', 'CTLA4-NA-z',
       'IL15RA-41BB-z', 'IL15RA-CD28-z', 'IL15RA-CD40-z',
       'IL15RA-CTLA4-z', 'IL15RA-IL15RA-z', 'IL15RA-NA-z', 'NA-NA-NA',
       'NA-NA-z', 'sel_cars_mixed', '41BB_OOD_cars_mixed',
       'all_cars_mixed'], dtype=object)

In [12]:
cd4

AnnData object with n_obs × n_vars = 36717 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'CAR_Variant', 'Donor', 'Time', 'Lane', 'Sample', 'percent.mt', 'A', 'B', 'C', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'RNA_snn_res.0.75', 'seurat_clusters', 'subset_RNA', 'subset_ADT', 'subset', 'Cytotoxicity_1', 'Proinflamatory_2', 'Memory_3', 'CD4_Th1_4', 'CD4_Th2_5', 'ident', 'time_subset'
    obsm: 'HARMONY', 'X_pca', 'X_umap'
    layers: 'logcounts'

### CD8

In [13]:
cd8

AnnData object with n_obs × n_vars = 24624 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'CAR_Variant', 'Donor', 'Time', 'Lane', 'Sample', 'percent.mt', 'A', 'B', 'C', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'RNA_snn_res.0.75', 'seurat_clusters', 'subset_RNA', 'subset_ADT', 'subset', 'Cytotoxicity_1', 'Proinflamatory_2', 'Memory_3', 'CD4_Th1_4', 'CD4_Th2_5', 'ident', 'time_subset'
    obsm: 'HARMONY', 'X_pca', 'X_umap'
    layers: 'logcounts'

In [14]:
min_cells = cd8.obs.groupby("CAR_Variant").size().min()
for car in cars:
    sample = sc.pp.sample(cd8, n=min_cells, replace=False, copy=True, rng=1234)
    if car in cd8_cars:
        sample.obs["CAR_Variant"] = "sel_cars_mixed"
        cd8 = ad.concat([cd8, sample])
    if "41BB" in car:
        sample.obs["CAR_Variant"] = "41BB_OOD_cars_mixed"
        cd8 = ad.concat([cd8, sample])
    sample.obs["CAR_Variant"] = "all_cars_mixed"
    cd8 = ad.concat([cd8, sample])
cd8.obs_names_make_unique()

/var/folders/02/gztwz1tj1051mtt_ykv6v7940000gn/T/ipykernel_6316/2480507459.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  min_cells = cd8.obs.groupby("CAR_Variant").size().min()
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To mak

In [15]:
min_cells

122

In [13]:
cd8.obs["CAR_Variant"].unique()

array(['41BB-41BB-z', '41BB-CD28-z', '41BB-CD40-z', '41BB-CTLA4-z',
       '41BB-IL15RA-z', '41BB-NA-z', 'CD28-41BB-z', 'CD28-CD28-z',
       'CD28-CD40-z', 'CD28-CTLA4-z', 'CD28-IL15RA-z', 'CD28-NA-z',
       'CD40-41BB-z', 'CD40-CD28-z', 'CD40-CD40-z', 'CD40-CTLA4-z',
       'CD40-IL15RA-z', 'CD40-NA-z', 'CTLA4-41BB-z', 'CTLA4-CD28-z',
       'CTLA4-CD40-z', 'CTLA4-CTLA4-z', 'CTLA4-IL15RA-z', 'CTLA4-NA-z',
       'IL15RA-41BB-z', 'IL15RA-CD28-z', 'IL15RA-CD40-z',
       'IL15RA-CTLA4-z', 'IL15RA-IL15RA-z', 'IL15RA-NA-z', 'NA-NA-NA',
       'NA-NA-z', 'sel_cars_mixed', '41BB_OOD_cars_mixed',
       'all_cars_mixed'], dtype=object)

In [14]:
cd8

AnnData object with n_obs × n_vars = 31212 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'CAR_Variant', 'Donor', 'Time', 'Lane', 'Sample', 'percent.mt', 'A', 'B', 'C', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'RNA_snn_res.0.75', 'seurat_clusters', 'subset_RNA', 'subset_ADT', 'subset', 'Cytotoxicity_1', 'Proinflamatory_2', 'Memory_3', 'CD4_Th1_4', 'CD4_Th2_5', 'ident', 'time_subset'
    obsm: 'HARMONY', 'X_pca', 'X_umap'
    layers: 'logcounts'

In [15]:
cd4.write_h5ad("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD4_with_dummy_cars.h5ad")
cd8.write_h5ad("/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CD8_with_dummy_cars.h5ad")